# Chapter 3 Feature Engineering for Marketing Data

In the previous chapter, we learned how to preprocess and explore our marketing data using Python libraries like Pandas, Matplotlib, and Seaborn. Now, we'll dive into feature engineering, a crucial step in the data science workflow that involves creating new and informative features from existing data to improve the predictive power of our models.

Feature engineering is particularly important in marketing analytics, as it helps us capture more complex patterns and relationships in customer behavior and market trends. In this chapter, we'll explore various techniques for creating new features using Python and apply them to our Online Retail dataset.

### 3.1 Recency, Frequency, and Monetary (RFM) Analysis
One of the most common feature engineering techniques in marketing is Recency, Frequency, and Monetary (RFM) analysis. RFM analysis segments customers based on three key metrics:

- Recency: How recently a customer made a purchase
- Frequency: How often a customer makes purchases
- Monetary: How much a customer spends

Let's calculate RFM metrics for our Online Retail dataset using Python:

In [5]:
import pandas as pd

# read the csv file into a pandas dataframe 
df = pd.read_csv('data\online_retail.csv')
df['Amount'] = df['Quantity'] * df['Price']
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Amount
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,30.0


In [8]:
import pandas as pd
from datetime import datetime

# Calculate the maximum InvoiceDate for each customer
max_date = df.groupby('Customer ID')['InvoiceDate'].max()

# Calculate Recency, Frequency, and Monetary metrics
rfm = df.groupby('Customer ID').agg({
    'InvoiceDate': lambda x: (datetime(2011, 12, 10) - x.max()).days,
    'Invoice': 'nunique',
    'Amount': 'sum'
})

# Rename columns
rfm.columns = ['Recency', 'Frequency', 'Monetary']

# Assign quintile ranks to each metric
rfm['R_rank'] = pd.qcut(rfm['Recency'], 5, labels=False, duplicates='drop')
rfm['F_rank'] = pd.qcut(rfm['Frequency'], 5, labels=False, duplicates='drop')
rfm['M_rank'] = pd.qcut(rfm['Monetary'], 5, labels=False, duplicates='drop')

# Add 1 to the ranks to start ranking from 1 instead of 0
rfm['R_rank'] += 1
rfm['F_rank'] += 1
rfm['M_rank'] += 1

# Combine R, F, and M ranks into a single RFM score
rfm['RFM_score'] = rfm['R_rank'].astype(str) + rfm['F_rank'].astype(str) + rfm['M_rank'].astype(str)

In [9]:
rfm.head()

,Recency,Frequency,Monetary,R_rank,F_rank,M_rank,RFM_score
Customer ID,,,,,,,
12346.0,325,17,-64.68,4,4,1,441
12347.0,2,8,5633.32,1,3,5,135
12348.0,75,5,2019.40,3,2,4,324
12349.0,18,5,4404.54,1,2,5,125
12350.0,310,1,334.40,4,1,2,412


In this code snippet, we first calculate the maximum InvoiceDate for each customer. Then, we calculate the Recency, Frequency, and Monetary metrics using the groupby() and agg() functions. We assign quintile ranks to each metric using pd.qcut() and combine the ranks into a single RFM score.

The resulting RFM score can be used to segment customers and tailor marketing strategies accordingly. For example, customers with high RFM scores (e.g., 555) are likely to be high-value customers who have made recent, frequent, and high-value purchases.


By calculating RFM metrics for the Online Retail dataset, we can segment our customers based on their purchasing behavior and create targeted marketing campaigns. For example, we might offer incentives to customers in the lowest recency quintiles (Q1 or Q2) to encourage them to make a purchase. Similarly, we might target customers in the highest monetary quintiles (Q4 or Q5) with high-value products or services.

Now that we've calculated RFM metrics for our dataset, let's move on to other feature engineering techniques for marketing data.


## 3.2 Time-based Features

Time-based features can provide valuable insights into customer behavior over time. For example, we might want to understand how a customer's purchasing behavior changes throughout the year or the impact of a marketing campaign on sales over time.

To create time-based features for our Online Retail dataset, we might use techniques such as:

* Rolling averages and moving averages to capture trends in customer behavior over time
* Lagged features to capture the impact of previous purchases or events on current behavior
* Year, month, day, and hour-of-the-day features to capture seasonality or time-specific patterns

Let's create time-based features for our Online Retail dataset:

In [11]:
# Extract time-based features from InvoiceDate
df['InvoiceYear'] = df['InvoiceDate'].dt.year
df['InvoiceMonth'] = df['InvoiceDate'].dt.month
df['InvoiceDay'] = df['InvoiceDate'].dt.day
df['InvoiceDayOfWeek'] = df['InvoiceDate'].dt.dayofweek
df['InvoiceHour'] = df['InvoiceDate'].dt.hour

# Calculate the time since last purchase for each customer
df['TimeSinceLastPurchase'] = df.groupby('Customer ID')['InvoiceDate'].diff()
df['TimeSinceLastPurchase'] = df['TimeSinceLastPurchase'].dt.days

# Calculate the time between first and last purchase for each customer
first_purchase = df.groupby('Customer ID')['InvoiceDate'].min()
last_purchase = df.groupby('Customer ID')['InvoiceDate'].max()
df['CustomerLifetime'] = (last_purchase - first_purchase).dt.days



In [ ]:
# Convert the InvoiceDate column to datetime type
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Set the index of the dfFrame to InvoiceDate
df.set_index('InvoiceDate', inplace=True)

# Create a rolling average of sales for each customer over the last 30 days
customer_sales = df.groupby('CustomerID')['Amount'].rolling(30).mean()
customer_sales.head(10)


In [ ]:

# Create lagged features for each customer's total sales and number of orders
customer_df = df.groupby('CustomerID').agg({'Amount': 'sum', 'Invoice': 'nunique'}).reset_index()
customer_df['LaggedAmount'] = customer_df['Amount'].shift(1)
customer_df['LaggedInvoiceNo'] = customer_df['Invoice'].shift(1)

# Create year, month, day, and hour-of-the-day features
customer_df['Year'] = df['InvoiceDate'].dt.year
customer_df['Month'] = df['InvoiceDate'].dt.month
customer_df['Day'] = df['InvoiceDate'].dt.day
customer_df['DayOfWeek'] = df['InvoiceDate'].dt.dayofweek
customer_df['Hour'] = df['InvoiceDate'].dt.hour

Holiday Features

In [ ]:
import pandas as pd
import holidays

# Assume 'data' is our preprocessed Online Retail dataset

# Create a holiday object for the United Kingdom
us_holidays = holidays.US()  # this is a dict

# Create a feature indicating if a transaction occurred on a holiday
df['Holiday'] = df['InvoiceDate'].apply(lambda x: x.date() in us_holidays)

# Create a feature indicating the name of the holiday (if applicable)
df['HolidayName'] = df['InvoiceDate'].apply(lambda x: uk_holidays.get(x.date()))

# Create a feature indicating if a transaction occurred on a weekend
df['Weekend'] = df['InvoiceDate'].dt.dayofweek.isin([5, 6])

# Calculate the total sales on holidays and weekends for each customer
df['HolidaySales'] = df.groupby(['CustomerID', 'Holiday'])['Amount'].transform('sum')
df['WeekendSales'] = df.groupby(['CustomerID', 'Weekend'])['Amount'].transform('sum')

In this example, we extract various time-based features from the InvoiceDate column, such as the year, month, day, day of the week, and hour of each transaction. We also calculate the time since the last purchase for each customer using the diff() function and the time between the first and last purchase (customer lifetime) using the min() and max() functions.

These time-based features can provide valuable insights into customer purchasing patterns and help us create more targeted marketing campaigns.

## 3.3 Product-based Features
Product-based features capture information about the products customers purchase and their characteristics. Let's create some product-based features for our Online Retail dataset:

In [13]:
# Calculate the total number of unique products purchased by each customer
df['NumUniqueProduts'] = df.groupby('Customer ID')['StockCode'].nunique()
 
# Calculate the average unit price of products purchased by each customer
df['AvgUnitPrice'] = df.groupby('Customer ID')['Price'].mean()

# Calculate the total quantity of products purchased by each customer
df['TotalQuantity'] = df.groupby('Customer ID')['Quantity'].sum()

# Calculate the average quantity per transaction for each customer
df['AvgQuantityPerTransaction'] = df.groupby('Customer ID')['Quantity'].mean()

In this code snippet, we calculate various product-based features such as the total number of unique products purchased by each customer, the average unit price of products purchased, the total quantity of products purchased, and the average quantity per transaction.

These features can help us understand customer preferences and purchasing habits, which can inform product recommendations, cross-selling, and upselling strategies.


## 3.4 Engineering Interaction Features 

Interaction features capture the relationship between two or more variables in our dataset. For example, we might want to understand how the relationship between a customer's total sales and the number of orders they place impacts their overall value to the business.

To create interaction features for our Online Retail dataset, we can use techniques such as:

* Multiplying or dividing features to create new features that capture the relationship between them
* Polynomial features to capture more complex relationships between variables

Let's create interaction features for our Online Retail dataset:


In [ ]:
# Create an interaction feature between total sales and the number of orders
customer_df['SalesPerOrder'] = customer_df['Amount'] / customer_df['Invoice']

# Use polynomial features to capture more complex relationships between variables
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures()
poly_features = pd.DataFrame(poly.fit_transform(customer_df[['LaggedAmount', 'LaggedInvoiceNo']]))
poly_features.columns = ['LaggedAmount^2', 'LaggedAmount * LaggedInvoiceNo', 'LaggedInvoiceNo^2']
customer_df = pd.concat([customer_df, poly_features], axis=1)

By creating interaction features for our Online Retail dataset, we can capture more complex patterns and relationships in customer behavior and market trends.


## 3.4 Conclusion
In this chapter, we explored feature engineering techniques for marketing data using Python. We created new features based on RFM analysis, time-based patterns, and product characteristics to capture more complex relationships in our data.

Feature engineering is an iterative process that requires domain knowledge, creativity, and experimentation. By creating informative and relevant features, we can improve the performance of our marketing analytics models and gain deeper insights into customer behavior.

In the next chapter, we'll dive into predictive analytics and learn how to build machine learning models for marketing tasks such as customer segmentation, churn prediction, and sales forecasting.